# ANOVA

In [30]:
import numpy as np
from scipy import stats

### ANOVA

##### One-way ANOVA

In [31]:
def anova1_partition_tss(X):
    """
    Partition the total sum of squares (ss_total) into between-group sum of squares (ss_b) and within-group sum of squares (ss_w).

    Parameters:
    -----------
    X : array_like
        A 2D array where each row represents a group and each column represents an observation.

    Returns:
    --------
    ss_total : float
        Total sum of squares.
    ss_w : float
        Within-group sum of squares.
    ss_b : float
        Between-group sum of squares.
    """

    # Flatten the input array and calculate the total mean
    all_data = np.concatenate(X)
    grand_mean = np.mean(all_data)  # X_bar

    # Calculate the total sum of squares
    ss_total = np.sum((all_data - grand_mean) ** 2)

    # Calculate the means and sizes of each group
    group_means = [np.mean(group) for group in X]
    group_sizes = [len(group) for group in X]

    # Calculate the between-group sum of squares
    ss_b = np.sum([size * (group_mean - grand_mean) ** 2 for group_mean, size in zip(group_means, group_sizes)])

    # Calculate the within-group sum of squares
    ss_w = np.sum([np.sum((group - group_mean) ** 2) for group, group_mean in zip(X, group_means)])

    return ss_total, ss_w, ss_b

In [32]:
def anova1_test_equality(X, alpha=0.05):
    """
    Tests the equality of means in one-way ANOVA.

    Parameters:
    -----------
    X : array_like
        A 2D array where each row represents a group and each column represents an observation.
    alpha : float, optional
        Significance level for the test. Default is 0.05.

    Returns:
    --------
    None, prints the ANOVA table, critical value, p-value, and whether to reject the null hypothesis.
    """
    # Number of groups
    I = len(X)

    # Total number of observations
    N = sum(len(group) for group in X)

    # Degrees of freedom
    df_b = I - 1  # Between-group degrees of freedom
    df_w = N - I  # Within-group degrees of freedom
    df_total = N - 1  # Total degrees of freedom

    # Partition the total sum of squares
    ss_total, ss_w, ss_b = anova1_partition_tss(X)

    # Calculate the mean squares
    ms_b = ss_b / df_b  # Mean square between
    ms_w = ss_w / df_w  # Mean square within

    # Calculate the F-statistic
    F = ms_b / ms_w

    # Calculate the critical value from the F-distribution
    f_critical = stats.f.ppf(1 - alpha, df_b, df_w)

    # Calculate the p-value
    p_value = 1 - stats.f.cdf(F, df_b, df_w)

    # Print the ANOVA table
    print("\nANOVA Table")
    print("-" * 80)
    print(f"{'Source':<15} {'df':<10} {'SS':<15} {'MS':<15} {'F':<15}")
    print("-" * 80)
    print(f"{'Between':<15} {df_b:<10} {ss_b:<15.4f} {ms_b:<15.4f} {F:<15.4f}")
    print(f"{'Within':<15} {df_w:<10} {ss_w:<15.4f} {ms_w:<15.4f}")
    print(f"{'Total':<15} {df_total:<10} {ss_total:<15.4f}")
    print("-" * 80)
    print(f"Critical value (f_{alpha}_{df_b}_{df_w}): {f_critical:.4f}")
    print(f"p-value: {p_value:.8f}")

    # Decision
    decision = "Reject H0" if F > f_critical else "Fail to reject H0"
    interpretation = ("There is signifant evidence to suggest that at least one group mean is different."
                      if decision == "Reject H0"
                      else "There is not enough evidence to suggest that group means are different.")

    print(f"Decision ({alpha=}): {decision}")
    print(f"Interpretation: {interpretation}")

    return {
        'F': F,
        'p_value': p_value,
        'f_critical': f_critical,
        'SS_between': ss_b,
        'SS_within': ss_w,
        'SS_total': ss_total,
        'df_between': df_b,
        'df_within': df_w,
        'df_total': df_total,
        'MS_between': ms_b,
        'MS_within': ms_w,
        'decision': decision,
    }

In [33]:
def anova1_is_contrast(c):
    """
    Determines if a linear combination defined by coefficients is a contrast.
    A contrast is a linear combination where the sum of coefficients is zero.

    Parameters:
    -----------
    c: list or numpy array
        Coefficients defining the linear combination

    Returns:
    --------
    bool
        True if the linear combination is a contrast, False otherwise
    """
    return abs(sum(c)) < 1e-10  # Using a small tolerance for floating-point arithmetic

In [34]:
def anova1_is_orthogonal(group_sizes, coef1, coef2):
    """
    Determines if two contrasts are orthogonal.
    Contrasts are orthogonal if the sum of (c1_i * c2_i / n_i) is zero.

    Parameters:
    -----------
    group_sizes: list or np array
        Sizes of each group
    coef1: list or np array
        Coefficients of the first linear combination
    coef2: list or np array
        Coefficients of the second linear combination

    Returns:
    --------
    bool
        True if the contrasts are orthogonal, False otherwise
    """
    # Check if both are contrasts
    if not anova1_is_contrast(c=coef1):
        print("Warning: The first linear combination is not a contrast")
        return False
    
    if not anova1_is_contrast(c=coef2):
        print("Warning: The second linear combination is not a contrast")
        return False
    
    # Check orthogonality
    orthogonality_sum = sum(c1 * c2 / n for c1, c2, n in zip(coef1, coef2, group_sizes))
    return abs(orthogonality_sum) < 1e-10 # Using a small tolerance for floating-point arithmetic

In [35]:
def bonferroni_correction(alpha, m):
    """
    Determines the significance level with Bonferroni correction.

    Parameters:
    -----------
    alpha: float
        Family-wise error rate (FWER)
    m: int
        Number of tests

    Returns:
    --------
    float
        Individual test significance level
    """
    return alpha / m

In [36]:
def sidak_correction(alpha, m):
    """
    Determines the significance level with Sidak correction.

    Parameters:
    -----------
    alpha: float
        Family-wise error rate (FWER)
    m: int
        Number of tests

    Returns:
    --------
    float
        Individual test significance level
    """
    return 1 - (1 - alpha) ** (1 / m)

In [37]:
# Check if all are pairwise comparisons (of the form [0,0,1,-1,0,0])
def is_pairwise(c):
    return sum(1 for x in c if abs(x) > 1e-10) == 2 and any(abs(abs(x) - 1) < 1e-10 for x in c) and sum(c) == 0

In [38]:
def anova1_ci_linear_combs(X, alpha, C, method="best"):
    """
    Computes simultaneous confidence intervals for linear combinations of group means.

    Parameters:
    -----------
    X: list of lists or numpy arrays
        Each inner list/array contains observations for one group
    alpha: float
        Significance level
    C: np array
        An m x I matrix where each row defines a linear combination of the group means
    method: str
        Method for confidence intervals: "Scheffe", "Tukey", "Bonferroni", "Sidak" or "best"

    Returns:
    --------
    list of tuples
        Confidence intervals for each linear combination
    method:
        The actual method used to calculate CI
    """
    # Number of groups and total observations
    I = len(X)
    group_sizes = [len(group) for group in X]
    N = sum(group_sizes)    # sum of all n_i
    m = C.shape[0]  # Number of linear combinations

    # Compute group means
    group_means = np.array([np.mean(group) for group in X])

    # Calculate within-group mean square (MSE)
    _, ss_w, _ = anova1_partition_tss(X=X)
    df_w = N - I
    mse = ss_w / df_w

    # Determine if all combinations are contrasts
    all_contrasts = all(anova1_is_contrast(c=C[i]) for i in range(m))

    all_pairwise = all(is_pairwise(c) for c in C)

    # Check orthogonality of contrasts (check all contrasts pairwise) 
    are_orthogonal = True
    if all_contrasts and m > 1:
        for i in range(m):
            for j in range(i+1, m):
                if not anova1_is_orthogonal(group_sizes=group_sizes, coef1=C[i], coef2=C[j]):
                    # Found a pair which are not orthogonal
                    are_orthogonal = False
                    break
            if not are_orthogonal:
                break
    
    # Calculate the estimate of each linear combination, sum of c_i * X_i bar
    estimates = [np.sum(c * group_means) for c in C]

    # Select the appropriate method if "best" is chosen
    if method == "best":
        if all_contrasts:
            if are_orthogonal:
                # Case: 4
                if all_pairwise:
                    # All orthogonal and pairwise: Thm 2.11 (Tukey CI for pairwise differences) vs Sidak
                    tukey_q = stats.studentized_range.ppf(1 - alpha, I, df_w)
                    sidak_alpha = sidak_correction(alpha=alpha, m=m)

                    # Calculate t factor
                    sidak_t = stats.t.ppf(1 - sidak_alpha/2, df_w)

                    method = "Tukey" if tukey_q / np.sqrt(2) < sidak_t else "Sidak"
                # Case: 1
                else:
                    # Orthogonal contrasts: Thm 2.7 (Scheffe's thm for contrasts) vs Sidak
                    sidak_alpha = sidak_correction(alpha=alpha, m=m)

                    # Calculate t and M factors
                    sidak_t = stats.t.ppf(1 - sidak_alpha/2, df_w)
                    scheffe_m = np.sqrt((I - 1) * stats.f.ppf(1 - alpha, I - 1, df_w))

                    method = "Sidak" if sidak_t < scheffe_m else "Scheffe"
            # Case: 3
            elif all_pairwise:
                # All pairwise: Thm 2.11 (Tukey CI for pairwise differences) vs Bonferroni
                tukey_q = stats.studentized_range.ppf(1 - alpha, I, df_w)
                bonferroni_alpha = bonferroni_correction(alpha=alpha, m=m)

                # Calculate t factor
                bonferroni_t = stats.t.ppf(1 - bonferroni_alpha/2, df_w)

                method = "Tukey" if tukey_q / np.sqrt(2) < bonferroni_t else "Bonferroni"
            # Case: 2
            else:
                # Non-orthogonal contrasts: Thm 2.7 (Scheffe's thm for contrasts) vs Bonferroni
                scheffe_m = np.sqrt((I - 1) * stats.f.ppf(1 - alpha, I - 1, df_w))
                bonferroni_alpha = bonferroni_correction(alpha=alpha, m=m)

                bonferroni_t = stats.t.ppf(1 - bonferroni_alpha/2, df_w)

                method = "Bonferroni" if bonferroni_t < scheffe_m else "Scheffe"
        # Case: 5
        else:
            # Not all contrasts: Thm 2.6 (Scheffe's thm for linear combinations) vs Bonferroni
            scheffe_m = np.sqrt(I * stats.f.ppf(1 - alpha, I, df_w))
            bonferroni_alpha = bonferroni_correction(alpha=alpha, m=m)

            bonferroni_t = stats.t.ppf(1 - bonferroni_alpha/2, df_w)
            
            method = "Bonferroni" if bonferroni_t < scheffe_m else "Scheffe"

    # Calculate confidence intervals based on the selected method
    intervals = []
    
    if method == "Scheffe":
        if all_contrasts:
            # Apply Thm 2.7 (Scheffe's thm for contrasts)
            scheffe_m = np.sqrt((I - 1) * stats.f.ppf(1 - alpha, I - 1, df_w))
        else:
            # Apply Thm 2.6 (Scheffe's thm for linear combinations)
            scheffe_m = np.sqrt(I * stats.f.ppf(1 - alpha, I, df_w))

        for i, c in enumerate(C):
            half_width = scheffe_m * np.sqrt((ss_w / df_w) * sum(c**2 / group_sizes))
            intervals.append((estimates[i] - half_width, estimates[i] + half_width))
    
    elif method == "Tukey":
        if not all_pairwise:
            print("Warning: Tukey's method is only valid for pairwise comparisons.")
            return None
        
        tukey_q = stats.studentized_range.ppf(1 - alpha, I, df_w)
        
        for i, c in enumerate(C):
            # Find 2 non-zero components
            idx1, idx2 = [j for j, val in enumerate(c) if abs(val) > 1e-10]
            half_width = (tukey_q / np.sqrt(2)) * np.sqrt((ss_w / df_w) * (1 / group_sizes[idx1] + 1 / group_sizes[idx2]))
            intervals.append((estimates[i] - half_width, estimates[i] + half_width))
    
    elif method == "Bonferroni":
        bonferroni_alpha = bonferroni_correction(alpha=alpha, m=m)
        bonferroni_t = stats.t.ppf(1 - bonferroni_alpha/2, df_w)

        for i, c in enumerate(C):
            half_width = bonferroni_t * np.sqrt((ss_w/df_w) * sum(c**2 / group_sizes))
            intervals.append((estimates[i] - half_width, estimates[i] + half_width))
    
    elif method == "Sidak":
        if not all_contrasts:
            print("Warning: Sidak's method is only valid for contrasts in this implementation")
            return None
        
        sidak_alpha = sidak_correction(alpha=alpha, m=m)
        sidak_t = stats.t.ppf(1 - sidak_alpha/2, df_w)

        for i, c in enumerate(C):
            half_width = sidak_t * np.sqrt((ss_w / df_w) * sum(c**2 / group_sizes))
            intervals.append((estimates[i] - half_width, estimates[i] + half_width))

    else:
        print(f"Error: Method '{method}' not recognized.")
        return None
    
    print(f"Using method: {method}")
    return intervals, method

In [39]:
def anova1_test_linear_combs(X, alpha, C, d, method="best"):
    """
    Performs simultaneous hypothesis tests for linear combinations of group means.

    Parameters:
    -----------
    X: list of lists or numpy arrays
        Each inner list/array contains observations for one group
    alpha: float
        Family-Wise Error Rate (FWER)
    C: np.array
        An m x I matrix where each row defines a linear combination of the group means
    d: np array
        An m x 1 vector of hypothesized values for each linear combination
    method: str
        Method for hypothesis testing: "Scheffe", "Tukey", "Bonferroni", "Sidak" or "best"

    Returns:
    --------
    dict
        A dictionary containing:
        - 'reject': boolean array indicating which hypotheses to reject
        - 'p_values': p-values for each hypothesis test
    """

    # Get the CI and the method used
    confidence_intervals, actual_method = anova1_ci_linear_combs(X=X, alpha=alpha, C=C, method=method)

    # Number of groups and total observations
    I = len(X)
    group_sizes = [len(group) for group in X]
    N = sum(group_sizes)
    m = C.shape[0]  # Number of linear combinations

    # Compute group means
    group_means = np.array([np.mean(group) for group in X])

    # Calculate within-group mean square (MSE)
    _, ss_w, _ = anova1_partition_tss(X=X)
    df_w = N - I
    mse = ss_w / df_w

    # Calculate the estimate of each linear combination
    estimates = [np.sum(c * group_means) for c in C]

    # Determine if all combinations are contrasts (to differentiate M factors in Thm 2.6 & 2.7)
    all_contrasts = all(anova1_is_contrast(c=C[i]) for i in range(m))

    # Determine rejection decisions based on CI
    reject = np.zeros(m, dtype=bool)
    p_values = np.zeros(m)

    # For each linear combination
    for i in range(m):
        # Reject H0, if d[i] is outside the CI
        reject[i] = not (confidence_intervals[i][0] <= d[i] <= confidence_intervals[i][1])

        # Calculate the test statistic
        diff = estimates[i] - d[i]
        t_stat = abs(diff) / np.sqrt(mse * sum(C[i]**2 / group_sizes))

        # Calculate p-value based on the actual method used
        if actual_method == "Scheffe":
            # Thm 2.7 (Scheffe's thm for contrasts)
            if all_contrasts:
                # For contrasts, compute F with I-1 numerator df
                # f_stat = abs_t_stat**2 * (I - 1)    # may be problematic
                f_stat = (diff ** 2) / ((I - 1) * mse * sum(C[i]**2 / group_sizes))
                p_values[i] = 1 - stats.f.cdf(f_stat, I-1, df_w)
            
            # Thm 2.6 (Scheffe's thm for linear combs)
            else:
                # For general linear combs, compute F with I numerator df
                # f_stat = abs_t_stat**2 * I  # may be problematic
                f_stat = (diff ** 2) / (I * mse * sum(C[i]**2 / group_sizes))
                p_values[i] = 1 - stats.f.cdf(f_stat, I, df_w)
        
        elif actual_method == "Tukey":
            # Convert t-statistic to studentized range statistic q
            q_stat = t_stat * np.sqrt(2)
            p_values[i] = 1 - stats.studentized_range.cdf(q_stat, I, df_w)
        
        elif actual_method == "Bonferroni":
            # Calculate p-value using t-distr and apply Bonferroni correction
            # No need to adjust alpha, adjust p_value itself
            p_value_uncorrected = 2 * (1 - stats.t.cdf(t_stat, df_w))   # t-distr is symmetric
            p_values[i] = min(p_value_uncorrected * m, 1.0)
        
        elif actual_method == "Sidak":
            # Calculate p-value using t-distribution and apply Sidak correction
            p_value_uncorrected = 2 * (1 - stats.t.cdf(t_stat, df_w))
            p_values[i] = 1 - (1 - p_value_uncorrected)**(1/m) if p_value_uncorrected < 1 else 1.0

    # Make sure reject decisions are consistent with p-values
    # This serves as a double-check - both should give the same result
    for i in range(m):
        if p_values[i] < alpha:
            if not reject[i]:
                print(f"Warning: Inconsistency detected for hypothesis {i+1}. "
                        f"p-value = {p_values[i]:.6f} < {alpha}, but CI contains d[{i}] = {d[i]}")

    return {
        'reject': reject,
        'p_values': p_values
    }


##### Two-way ANOVA

In [40]:
def anova2_partition_tss(X):
    """
    Partition the total sum of squares (ss_total) in a two-way ANOVA layout.

    Parameters:
    -----------
    X : array_like
        A 3D array where X[i,j,k] is the k-th observation in the (i,j)-th cell

    Returns:
    --------
    ss_total : float
        Total sum of squares.
    ss_a : float
        Sum of squares for factor A.
    ss_b : float
        Sum of squares for factor B.
    ss_ab : float
        Sum of squares for the interaction between factors A and B.
    ss_e : float
        Sum of squares for error.
    """

    # Get dimensions
    I, J, K = X.shape

    # Calculate cell means, row means, column means, and grand mean
    cell_means = np.mean(X, axis=2)
    row_means = np.mean(cell_means, axis=1)
    col_means = np.mean(cell_means, axis=0)
    grand_mean = np.mean(X)

    # Flatten the data to compute total sum of squares
    all_data = X.flatten()
    ss_total = np.sum((all_data - grand_mean) ** 2)

    # Calculate the sum of squares for factor A
    ss_a = J * K * np.sum((row_means - grand_mean) ** 2)

    # Calculate the sum of squares for factor B
    ss_b = I * K * np.sum((col_means - grand_mean) ** 2)

    # Calculate the sum of squares for the interaction between factors A and B
    ss_ab = K * np.sum((cell_means - np.outer(row_means, np.ones(J)) - 
                        np.outer(np.ones(I), col_means) + grand_mean) ** 2)
    
    # Calculate the sum of squares for error
    ss_e = 0.0
    for i in range(I):
        for j in range(J):
            ss_e += np.sum((X[i, j, :] - cell_means[i, j]) ** 2)

    return ss_total, ss_a, ss_b, ss_ab, ss_e


In [41]:
def anova2_mle(X):
    """
    Computes the maximum likelihood estimates (MLE) for parameters in a two-way ANOVA.

    Parameters:
    -----------
    X : array_like
        A 3D array where X[i,j,k] is the k-th observation in the (i,j)-th cell

    Returns:
    --------
    mu : float
        Estimate of the grand mean.
    a : np.array
        Estimates of the row effects (factor A).
    b : np.array
        Estimates of the column effects (factor B).
    delta : np.array
        Estimates of the interaction effects (factor A x factor B).
    """

    # Get dimensions
    I, J, K = X.shape

    # Calculate cell means, row means, column means, and grand mean
    cell_means = np.mean(X, axis=2)
    row_means = np.mean(cell_means, axis=1)
    col_means = np.mean(cell_means, axis=0)
    grand_mean = np.mean(X)

    # Calculate main effects
    a = row_means - grand_mean
    b = col_means - grand_mean

    # Calculate interaction effects
    delta = np.zeros((I, J))
    for i in range(I):
        for j in range(J):
            delta[i, j] = cell_means[i, j] - row_means[i] - col_means[j] + grand_mean

    return grand_mean, a, b, delta


In [42]:
def anova2_test_equality(X, alpha=0.05, test_type="A"):
    """
    Perform one of the basic three tests for two-way ANOVA.

    Parameters:
    -----------
    X : array_like
        A 3D array where X[i,j,k] is the k-th observation in the (i,j)-th cell
    alpha : float, optional
        Significance level for the test. Default is 0.05.
    test_type : str, optional
        Type of test to perform. Options are "A", "B", or "AB" for main effects A, B, or interaction AB. Default is "A".
    
    Returns:
    --------
    dict
        Test outcome with relevant statistics.
    """

    # Get dimensions
    I, J, K = X.shape

    # Partition the total sum of squares
    ss_total, ss_a, ss_b, ss_ab, ss_e = anova2_partition_tss(X)

    # Calculate degrees of freedom
    df_a = I - 1  # Degrees of freedom for factor A
    df_b = J - 1  # Degrees of freedom for factor B
    df_ab = (I - 1) * (J - 1)  # Degrees of freedom for interaction
    df_e = I * J * (K - 1)  # Degrees of freedom for error
    df_total = I * J * K - 1  # Total degrees of freedom

    # Calculate mean squares
    ms_a = ss_a / df_a  # Mean square for factor A
    ms_b = ss_b / df_b  # Mean square for factor B
    ms_ab = ss_ab / df_ab  # Mean square for interaction
    ms_e = ss_e / df_e  # Mean square for error

    # Calculate F-statistics
    F_a = ms_a / ms_e  # F-statistic for factor A
    F_b = ms_b / ms_e  # F-statistic for factor B
    F_ab = ms_ab / ms_e  # F-statistic for interaction

    # Calculate critical values from the F-distribution
    f_critical_a = stats.f.ppf(1 - alpha, df_a, df_e)
    f_critical_b = stats.f.ppf(1 - alpha, df_b, df_e)
    f_critical_ab = stats.f.ppf(1 - alpha, df_ab, df_e)

    # Calculate p-values
    p_value_a = 1 - stats.f.cdf(F_a, df_a, df_e)
    p_value_b = 1 - stats.f.cdf(F_b, df_b, df_e)
    p_value_ab = 1 - stats.f.cdf(F_ab, df_ab, df_e)

    # Print the ANOVA table
    print("\nTwo-Way ANOVA Table:")
    print("-" * 80)
    print(f"{'Source':<15} {'df':<10} {'SS':<15} {'MS':<15} {'F':<15}")
    print("-" * 80)

    if test_type == "A" or test_type == "all":
        print(f"{'Factor A':<15} {df_a:<10} {ss_a:<15.4f} {ms_a:<15.4f} {F_a:<15.4f}")
    if test_type == "B" or test_type == "all":
        print(f"{'Factor B':<15} {df_b:<10} {ss_b:<15.4f} {ms_b:<15.4f} {F_b:<15.4f}")
    if test_type == "AB" or test_type == "all":
        print(f"{'Interaction':<15} {df_ab:<10} {ss_ab:<15.4f} {ms_ab:<15.4f} {F_ab:<15.4f}")

    print(f"{'Error':<15} {df_e:<10} {ss_e:<15.4f} {ms_e:<15.4f}")
    print(f"{'Total':<15} {df_total:<10} {ss_total:<15.4f}")
    print("-" * 80)

    # Decision and interpretation
    if test_type == "A":
        f_stat = F_a
        f_critical = f_critical_a
        p_value = p_value_a
        h0_description = "a_1 = a_2 = ... = a_I"
    elif test_type == "B":
        f_stat = F_b
        f_critical = f_critical_b
        p_value = p_value_b
        h0_description = "b_1 = b_2 = ... = b_J"
    elif test_type == "AB":
        f_stat = F_ab
        f_critical = f_critical_ab
        p_value = p_value_ab
        h0_description = "all delta_ij = 0"

    decision = "Reject H0" if f_stat > f_critical else "Fail to reject H0"

    print(f"\nTesting H0: {h0_description}")
    print(f"Decision ({alpha=}): {decision}")
    print(f"p-value: {p_value:.4f}")
    print(f"Critical value (f): {f_critical:.4f}")

    if test_type == "A":
        return {
            'F': F_a,
            'p_value': p_value_a,
            'f_critical': f_critical_a,
            'SS': ss_a,
            'MS': ms_a,
            'df': df_a,
            'decision': decision
        }
    elif test_type == "B":
        return {
            'F': F_b,
            'p_value': p_value_b,
            'f_critical': f_critical_b,
            'SS': ss_b,
            'MS': ms_b,
            'df': df_b,
            'decision': decision
        }
    elif test_type == "AB":
        return {
            'F': F_ab,
            'p_value': p_value_ab,
            'f_critical': f_critical_ab,
            'SS': ss_ab,
            'MS': ms_ab,
            'df': df_ab,
            'decision': decision
        }

### Linear Regression

In [43]:
def mult_lr_least_squares(X, y):
    """
    Finds the least squares estimates for a multiple linear regression model.
    
    Parameters:
    -----------
    X : array_like
        n x (k+1) design matrix where the first column is all ones
    y : array_like
        n x 1 response vector

    Returns:
    --------
    beta_hat : np.array
        (k+1) x 1 vector of maximum likelihood estimates for the regression coefficients
    sigma2_hat : float
        Maximum likelihood estimate of the error variance
    sigma2_hat_unbiased : float
        Unbiased estimate of the error variance
    """

    # Number of observations and predictors
    n, k_plus_1 = X.shape

    # Check if X has full column rank
    if np.linalg.matrix_rank(X) < k_plus_1:
        raise ValueError("Design matrix X does not have full column rank.")
    
    # Calculate the least squares estimates
    beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y

    # Calculate the residuals
    residuals = y - X @ beta_hat

    # Calculate MLE of error variance
    sigma2_hat = np.sum(residuals**2) / n

    # Calculate unbiased estimate of error variance
    sigma2_hat_unbiased = np.sum(residuals**2) / (n - k_plus_1)

    return beta_hat, sigma2_hat, sigma2_hat_unbiased

In [44]:
def mult_lr_partition_tss(X, y):
    """
    Partitions the total sum of squares into regression and residual components in a multiple linear regression model.

    Parameters:
    -----------
    X : array_like
        n x (k+1) design matrix where the first column is all ones
    y : array_like
        n x 1 response vector

    Returns:
    --------
    ss_total : float
        Total sum of squares.
    ss_reg : float
        Regression sum of squares.
    ss_res : float
        Residual sum of squares.
    """

    # Calculate the least squares estimates
    beta_hat, _, _ = mult_lr_least_squares(X, y)

    # Calculate the fitted values
    y_hat = X @ beta_hat

    # Calculate mean of y
    y_mean = np.mean(y)

    # Calculate sums of squares
    ss_total = np.sum((y - y_mean) ** 2)
    ss_reg = np.sum((y_hat - y_mean) ** 2)
    ss_res = np.sum((y - y_hat) ** 2)

    return ss_total, ss_reg, ss_res

    

In [45]:
def mult_norm_lr_simul_ci(X, y, alpha=0.05):
    """
    Computes simultaneous confidence intervals for the regression coefficients in a multiple linear regression model.

    Parameters:
    -----------
    X : array_like
        n x (k+1) design matrix where the first column is all ones
    y : array_like
        n x 1 response vector
    alpha : float, optional
        Significance level for the confidence intervals. Default is 0.05.

    Returns:
    --------
    list of tuples
        100(1-alpha)% confidence intervals (lower, upper) for each regression coefficient.
    method : str
        The method used for the confidence intervals: "Bonferroni" or "Scheffe".
    """

    # Number of observations and predictors
    n, k_plus_1 = X.shape

    # Calculate the least squares estimates
    beta_hat, _, sigma2_hat_unbiased = mult_lr_least_squares(X, y)

    # Square root of the unbiased estimate of the error variance
    s_e = np.sqrt(sigma2_hat_unbiased)

    cov_matrix = np.linalg.inv(X.T @ X)

    # Calculate the critical value from the t-distribution
    bonf_alpha = bonferroni_correction(alpha=alpha, m=k_plus_1)
    t_critical = stats.t.ppf(1 - bonf_alpha / 2, n - k_plus_1)

    # Calculate Scheffe's M factor
    scheffe_m = np.sqrt(k_plus_1 * stats.f.ppf(1 - alpha, k_plus_1, n - k_plus_1))

    # Use the minimum of the two methods
    if t_critical <= scheffe_m:
        method = "Bonferroni"
        half_widths = t_critical * s_e * np.sqrt(np.diag(cov_matrix))
    else:
        method = "Scheffe"
        half_widths = scheffe_m * s_e * np.sqrt(np.diag(cov_matrix))

    # Calculate confidence intervals
    ci = []
    for i in range(k_plus_1):
        lower_bound = beta_hat[i] - half_widths[i]
        upper_bound = beta_hat[i] + half_widths[i]
        ci.append((lower_bound, upper_bound))

    return ci, method
    

In [46]:
def mult_norm_lr_cr(X, y, C, alpha=0.05):
    """
    Computes 100(1-alpha)% confidence region for a linear combination of regression coefficients in a multiple linear regression model.
    
    Parameters:
    -----------
    X : array_like
        n x (k+1) design matrix where the first column is all ones
    y : array_like
        n x 1 response vector
    C: array_like
        r x (k+1) matrix of coefficients for the linear combinations with rank r
    alpha : float, optional
        Significance level for the confidence region. Default is 0.05.

    Returns:
    --------
    dict
        Parameters specifying the confidence region:
        - 'center': center of the confidence region
        - 'shape_matrix': shape matrix of the confidence region
        - 'radius_squared': square of radius of the confidence region
    """

    # Number of observations and predictors
    n, k_plus_1 = X.shape

    r = C.shape[0]  # Number of linear combinations

    XtX_inv = np.linalg.inv(X.T @ X)

    # Shape matrix: C @ (X^T X)^-1 @ C^T
    shape_matrix = C @ XtX_inv @ C.T

    # Calculate the least squares estimates
    beta_hat, _, sigma2_hat_unbiased = mult_lr_least_squares(X, y)

    # Calculate center: C*beta_hat
    C_beta_hat = C @ beta_hat

    # Calculate the critical value from the F-distribution
    f_critical = stats.f.ppf(1 - alpha, r, n - k_plus_1)
    
    # Calculate the radius
    radius_squared = r * sigma2_hat_unbiased * f_critical

    return {
        'center': C_beta_hat,
        'shape_matrix': shape_matrix,
        'radius_squared': radius_squared,
    }



In [47]:
def mult_norm_lr_is_in_cr(X, y, C, c0, alpha=0.05):
    """
    Checks if a vector c0 is inside the confidence region for C*beta.
    
    Parameters:
    -----------
    X : array_like
        n x (k+1) design matrix where the first column is all ones
    y : array_like
        n x 1 response vector
    C: array_like
        r x (k+1) matrix of coefficients for the linear combinations with rank r
    c0: array_like
        r x 1 vector to check if it is in the confidence region
    alpha : float, optional
        Significance level for the confidence region. Default is 0.05.

    Returns:
    --------
    bool
        True if c0 is inside the confidence region, False otherwise.
    """

    # Number of observations and predictors
    n, k_plus_1 = X.shape

    r = C.shape[0]  # Number of linear combinations

    # Calculate the least squares estimates
    beta_hat, _, sigma2_hat_unbiased = mult_lr_least_squares(X, y)

    # Get the confidence region parameters
    cr_params = mult_norm_lr_cr(X=X, y=y, C=C, alpha=alpha)

    center = cr_params['center']
    shape_matrix = cr_params['shape_matrix']
    radius_squared = cr_params['radius_squared']

    diff = center - c0

    left_side = diff.T @ np.linalg.inv(shape_matrix) @ diff
    
    return left_side <= radius_squared

In [48]:
def mult_norm_lr_test_general(X, y, C, c0, alpha=0.05):
    """
    Tests the null hypothesis H0: C*beta = c0 vs H1: C*beta != c0 at significance level alpha.

    Parameters:
    -----------
    X : array_like
        n x (k+1) design matrix where the first column is all ones
    y : array_like
        n x 1 response vector
    C: array_like
        r x (k+1) matrix of coefficients for the linear combinations with rank r
    c0: array_like
        r x 1 vector of hypothesized values for the linear combinations
    alpha : float, optional
        Significance level for the test. Default is 0.05.

    Returns:
    --------
    dict
        Test outcome with relevant statistics.
        - 'reject': boolean indicating whether to reject the null hypothesis
        - 'p_value': p-value for the test
        - 'F_stat': F-statistic for the test
        - 'f_critical': critical value from the F-distribution
    """

    # Number of observations and predictors
    n, k_plus_1 = X.shape

    r = C.shape[0]  # Number of linear combinations

    # Calculate the least squares estimates
    beta_hat, _, sigma2_hat_unbiased = mult_lr_least_squares(X, y)

    # Check if c0 is in the confidence region
    in_cr = mult_norm_lr_is_in_cr(X=X, y=y, C=C, c0=c0, alpha=alpha)

    # H0 is rejected if c0 is not in the confidence region
    reject = not in_cr

    # Calculate the critical value from the F-distribution
    f_critical = stats.f.ppf(1 - alpha, r, n - k_plus_1)

    # Calculate F-statistic
    C_beta_hat = C @ beta_hat
    diff = C_beta_hat - c0
    XtX_inv = np.linalg.inv(X.T @ X)
    shape_matrix = C @ XtX_inv @ C.T
    left_side = diff.T @ np.linalg.inv(shape_matrix) @ diff
    F_stat = left_side / (r * sigma2_hat_unbiased)

    # Calculate p-value
    p_value = 1 - stats.f.cdf(F_stat, r, n - k_plus_1)

    return {
        'reject': reject,
        'p_value': p_value,
        'F_stat': F_stat,
        'f_critical': f_critical
    }

In [49]:
def mult_norm_lr_test_comp(X, y, j_indices, alpha=0.05):
    """
    Tests the null hypothesis H0: beta_j = 0 for j in j_indices vs H1: not H0.

    Parameters:
    -----------
    X : array_like
        n x (k+1) design matrix where the first column is all ones
    y : array_like
        n x 1 response vector
    j_indices: list of int
        Indices of the coefficients to test
    alpha : float, optional
        Significance level for the test. Default is 0.05.

    Returns:
    --------
    dict
        Test outcome with relevant statistics.
        - 'reject': boolean indicating whether to reject the null hypothesis
        - 'p_value': p-value for the test
        - 'F_stat': F-statistic for the test
        - 'f_critical': critical value from the F-distribution
    """

    # Number of observations and predictors
    n, k_plus_1 = X.shape

    # Create the contrast matrix C
    r = len(j_indices)
    C = np.zeros((r, k_plus_1))
    for i, j in enumerate(j_indices):
        C[i, j] = 1

    # Create the hypothesized value vector c0
    c0 = np.zeros(r)

    # Perform the general test
    test_result = mult_norm_lr_test_general(X=X, y=y, C=C, c0=c0, alpha=alpha)
    
    return test_result
    

In [50]:
def mult_norm_lr_test_linear_reg(X, y, alpha=0.05):
    """
    Tests the existence of LR at all, H0: beta_1 = beta_2 = ... = beta_k = 0 vs H1: not H0.

    Parameters:
    -----------
    X : array_like
        n x (k+1) design matrix where the first column is all ones
    y : array_like
        n x 1 response vector
    alpha : float, optional
        Significance level for the test. Default is 0.05.

    Returns:
    --------
    dict
        Test outcome with relevant statistics.
        - 'reject': boolean indicating whether to reject the null hypothesis
        - 'p_value': p-value for the test
        - 'F_stat': F-statistic for the test
        - 'f_critical': critical value from the F-distribution
    """

    # Number of observations and predictors
    n, k_plus_1 = X.shape

    j_indices = list(range(1, k_plus_1))  # Exclude the intercept term

    # Use the mult_norm_lr_test_comp function to test the null hypothesis
    test_result = mult_norm_lr_test_comp(X=X, y=y, j_indices=j_indices, alpha=alpha)

    return test_result

In [51]:
def mult_norm_lr_pred_ci(X, y, D, alpha=0.05, method="best"):
    """
    Computes simultaneous confidence intervals for d_i^T * beta for rows of D.

    Parameters:
    -----------
    X : array_like
        n x (k+1) design matrix where the first column is all ones
    y : array_like
        n x 1 response vector
    D : array_like
        m x (k+1) design matrix for the predictions
    alpha : float, optional
        Significance level for the confidence intervals. Default is 0.05.
    method : str, optional
        One of "Scheffe", "Bonferroni" or "best". Default is "best".

    Returns:
    --------
    dict
        - 'lower': lower bounds for each d_i^T * beta
        - 'upper': upper bounds for each d_i^T * beta
        - 'method': method used for the confidence intervals
    """

    # Number of observations and predictors
    n, k_plus_1 = X.shape

    m = D.shape[0]  # Number of predictions

    # Calculate the least squares estimates
    beta_hat, _, sigma2_hat_unbiased = mult_lr_least_squares(X, y)
    S_e  = np.sqrt(sigma2_hat_unbiased)

    XtX_inv = np.linalg.inv(X.T @ X)

    # Bonferroni critical value
    t_crit = stats.t.ppf(1 - alpha / (2 * m), n - k_plus_1)

    # Scheffe critical value
    scheffe_m = np.sqrt(k_plus_1 * stats.f.ppf(1 - alpha, k_plus_1, n - k_plus_1))

    lower_bounds = []
    upper_bounds = []

    for i in range(m):
        d = D[i, :].reshape(-1, 1)  # Reshape to column vector
        d_beta_hat = d.T @ beta_hat # 1 x 1 matrix
        d_beta_hat = d_beta_hat.item()   # Convert to scalar

        se = S_e * np.sqrt(d.T @ XtX_inv @ d)   # 1 x 1 matrix
        se = se.item()   # Convert to scalar
        
        if method == "Scheffe":
            half_width = scheffe_m * se
        elif method == "Bonferroni":
            half_width = t_crit * se
        elif method == "best":
            half_width = min(scheffe_m, t_crit) * se
            method_used = "Bonferroni" if half_width == t_crit * se else "Scheffe"
        else:
            raise ValueError("Invalid method. Choose 'Scheffe', 'Bonferroni', or 'best'.")

        lower_bounds.append(d_beta_hat - half_width)
        upper_bounds.append(d_beta_hat + half_width)
    
    return {
        'lower': np.array(lower_bounds),
        'upper': np.array(upper_bounds),
        'method': method_used if method == "best" else method
    }


In [52]:
def run_anova_examples():
    """
    Run examples of ANOVA functions.
    """
    print("\n" + "=" * 80)
    print(" ANOVA Functions Examples ".center(80, "="))
    print("="*80)

    # Create sample data for one-way ANOVA
    np.random.seed(42)
    group1 = np.random.normal(loc=10, scale=2, size=15) # Group 1, 15 datapoints from N(10, 2^2)
    group2 = np.random.normal(loc=12, scale=2, size=12) # Group 2, 12 datapoints from N(12, 2^2)
    group3 = np.random.normal(loc=8, scale=2, size=18)  # Group 3, 18 datapoints from N(8, 2^2)

    X_1 = [group1, group2, group3]


    # Example 1: ANOVA1 partition TSS
    print("\n1. ANOVA1_partition_tss Example:")
    ss_total, ss_w, ss_b = anova1_partition_tss(X_1)
    print(f"Total Sum of Squares (ss_total): {ss_total:.4f}")
    print(f"Within-group Sum of Squares (ss_w): {ss_w:.4f}")
    print(f"Between-group Sum of Squares (ss_b): {ss_b:.4f}")
    print(f"Verification: ss_total = ss_w + ss_b: {ss_total:.4f} = {ss_w + ss_b:.4f}")


    # Example 2: ANOVA1 test equality
    print("\n2. ANOVA1_test_equality Example:")
    result = anova1_test_equality(X_1, alpha=0.05)


    # Example 3: ANOVA1 is contrast
    print("\n3. ANOVA1_is_contrast Example:")
    contrast1 = [1, -1, 0]  # mu_1 - mu_2
    contrast2 = [1, 1, -2]  # mu_1 + mu_2 - 2mu_3
    non_contrast = [1, 1, 1]    # mu_1 + mu_2 + mu_3

    print(f"Is [1, -1, 0] a contrast? {anova1_is_contrast(c=contrast1)}")
    print(f"Is [1, 1, -2] a contrast? {anova1_is_contrast(c=contrast2)}")
    print(f"Is [1, 1, 1] a contrast? {anova1_is_contrast(c=non_contrast)}")


    # Example 4: ANOVA1 is orthogonal
    print("\n4. ANOVA1_is_orthogonal Example:")
    group_sizes = [len(group) for group in X_1]
    print(f"Group sizes: {group_sizes}")
    print(f"Are [1, -1, 0] and [1, 1, -2] orthogonal? {anova1_is_orthogonal(group_sizes=group_sizes, coef1=contrast1, coef2=contrast2)}")
    # print(f"Are [1, -1, 0] and [1, 1, 1] orthogonal? {anova1_is_orthogonal(group_sizes=group_sizes, coef1=contrast1, coef2=non_contrast)}")


    # Example 5: Bonferroni correction
    print("\n5. Bonferroni_correction Example:")
    alpha = 0.05
    m = 3
    print(f"Bonferroni correction for FWER={alpha} and m={m}: {bonferroni_correction(alpha=alpha, m=m):.4f}")


    # Example 6: Sidak correction
    print("\n6. Sidak_correction Example:")
    print(f"Sidak correction for FWER={alpha} and m={m}: {sidak_correction(alpha=alpha, m=m):.4f}")


    # Example 7: ANOVA1 CI linear combs
    print("\n7. ANOVA1_CI_linear_combs Example:")
    C = np.array([
        [1, -1, 0], # mu_1 - mu_2
        [1, 0, -1], # mu_1 - mu_3
        [0, 1, -1]  # mu_2 - mu_3
    ])

    print("Computing confidence intervals for pairwise differences using different methods:")

    print("\nScheffe's method:")
    intervals_scheffe, _ = anova1_ci_linear_combs(X=X_1, alpha=alpha, C=C, method='Scheffe')
    for i, (lower, upper) in enumerate(intervals_scheffe):
        print(f"CI for contrast {i+1}: ({lower:.4f}, {upper:.4f}) --> upper - lower = {upper - lower:.4f}")

    print("\nTukey's method:")
    intervals_tukey, _ = anova1_ci_linear_combs(X=X_1, alpha=alpha, C=C, method="Tukey")
    for i, (lower, upper) in enumerate(intervals_tukey):
        print(f"CI for contrast {i+1}: ({lower:.4f}, {upper:.4f}) --> upper - lower = {upper - lower:.4f}")

    print("\nBonferroni's method:")
    intervals_bonferroni, _ = anova1_ci_linear_combs(X=X_1, alpha=alpha, C=C, method="Bonferroni")
    for i, (lower, upper) in enumerate(intervals_bonferroni):
        print(f"CI for contrast {i+1}: ({lower:.4f}, {upper:.4f}) --> upper - lower = {upper - lower:.4f}")

    print("\nSidak's method:")
    intervals_sidak, _ = anova1_ci_linear_combs(X=X_1, alpha=alpha, C=C, method="Sidak")
    for i, (lower, upper) in enumerate(intervals_sidak):
        print(f"CI for contrast {i+1}: ({lower:.4f}, {upper:.4f}) --> upper - lower = {upper - lower:.4f}")
    
    print("\nBest method (should choose Tukey for pairwise differences):")
    intervals_best, _ = anova1_ci_linear_combs(X=X_1, alpha=alpha, C=C, method="best")
    for i, (lower, upper) in enumerate(intervals_best):
        print(f"CI for contrast {i+1}: ({lower:.4f}, {upper:.4f}) --> upper - lower = {upper - lower:.4f}")

    
    # Example 8: ANOVA1 test linear combs
    print("\n8. ANOVA1_test_linear_combs Example:")
    d = np.array([1, 2, -2]) # Testing that all differences are 0
    results = anova1_test_linear_combs(X=X_1, alpha=alpha, C=C, d=d, method="Sidak")
    print(f"alpha = {alpha}")
    print(f"d = {d}\n")
    for i in range(m):
        print(f"Test for contrast {i+1}:")
        print(f"p-value: {results['p_values'][i]:.6f}")
        print(f"Decision: {'Reject H0' if results['reject'][i] else 'Fail to reject H0'}")
        
        print("")

    
    # Create sample data for two-way ANOVA
    np.random.seed(42)
    I, J, K = 3, 2, 4   # 3 levels of factor A, 2 levels of factor B, and 4 replicates

    # Create effects
    mu = 10
    a_effects = np.array([-1, 0, 1])  # Effects for factor A
    b_effects = np.array([-0.5, 0.5])  # Effects for factor B

    # Create interaction effects
    ab_effects = np.array([
        [0.5, -0.5],
        [0, 0],
        [-0.5, 0.5]
    ])

    # Generate data
    X_2 = np.zeros((I, J, K))
    for i in range(I):
        for j in range(J):
            cell_mean = mu + a_effects[i] + b_effects[j] + ab_effects[i, j]
            X_2[i, j, :] = np.random.normal(loc=cell_mean, scale=1, size=K) # Normal noise with sd=1


    # Example 9: ANOVA2 partition TSS
    print("\n9. ANOVA2_partition_tss Example:")
    ss_total, ss_a, ss_b, ss_ab, ss_e = anova2_partition_tss(X_2)
    print(f"Total Sum of Squares (ss_total): {ss_total:.4f}")
    print(f"Factor A Sum of Squares (ss_a): {ss_a:.4f}")
    print(f"Factor B Sum of Squares (ss_b): {ss_b:.4f}")
    print(f"Interaction Sum of Squares (ss_ab): {ss_ab:.4f}")
    print(f"Error Sum of Squares (ss_e): {ss_e:.4f}")
    print(f"Verification: ss_total = ss_a + ss_b + ss_ab + ss_e: {ss_total:.4f} = {ss_a + ss_b + ss_ab + ss_e:.4f}")


    # Example 10: ANOVA2 MLE
    print("\n10. ANOVA2_mle Example:")
    mu_hat, a_hat, b_hat, delta_hat = anova2_mle(X_2)
    print(f"Estimated grand mean (mu): {mu_hat:.4f}")
    print(f"Estimated row effects (a): {a_hat}")
    print(f"Estimated column effects (b): {b_hat}")
    print(f"Estimated interaction effects (delta):\n{delta_hat}")


    # Example 11: ANOVA2 test equality
    print("\n11. ANOVA2_test_equality Example:")
    
    print("\nTesting factor A effects:")
    anova2_test_equality(X_2, alpha=0.05, test_type="A")

    print("\nTesting factor B effects:")
    anova2_test_equality(X_2, alpha=0.05, test_type="B")

    print("\nTesting interaction effects:")
    anova2_test_equality(X_2, alpha=0.05, test_type="AB")

In [53]:
def run_regression_examples():
    """ Run examples of regression functions. """

    print("\n" + "=" * 80)
    print(" Linear Regression Functions Examples ".center(80, "="))
    print("="*80)

    # Create sample data for multiple linear regression
    np.random.seed(42)
    n = 100
    k_plus_1 = 4    # 3 predictors + intercept

    # True coefficients
    beta_true = np.array([2, 1.5, -0.5, 0.8])

    # Design matrix (X) with intercept
    X = np.ones((n, k_plus_1))
    X[:, 1] = np.random.uniform(-2, 2, n)  # Predictor 1
    X[:, 2] = np.random.uniform(-3, 3, n)  # Predictor 2
    X[:, 3] = np.random.uniform(-1, 1, n)  # Predictor 3

    # Response variable (y) with noise
    sigma = 1.0 # Standard deviation of error
    error = np.random.normal(0, sigma, n)
    y = X @ beta_true + error

    # Example 1: Mult_LR_least_squares
    print("\n1. Mult_LR_Least_squares Example:")
    beta_hat, sigma2_hat, sigma2_hat_unbiased = mult_lr_least_squares(X, y)
    print(f"True beta: {beta_true}")
    print(f"Estimated beta: {beta_hat}")
    print(f"\nTrue sigma^2: {sigma**2:.4f}")
    print(f"MLE of sigma^2: {sigma2_hat:.4f}")
    print(f"Unbiased estimate of sigma^2: {sigma2_hat_unbiased:.4f}")

    # Example 2: Mult_LR_partition_tss
    print("\n2. Mult_LR_partition_TSS Example:")
    ss_total, ss_reg, ss_res = mult_lr_partition_tss(X, y)
    print(f"Total Sum of Squares (ss_total): {ss_total:.4f}")
    print(f"Regression Sum of Squares (ss_reg): {ss_reg:.4f}")
    print(f"Residual Sum of Squares (ss_res): {ss_res:.4f}")
    print(f"Verification: ss_total = ss_reg + ss_res: {ss_total:.4f} = {ss_reg + ss_res:.4f}")
    
    # Example 3: Mult_norm_LR_simul_CI
    print("\n3. Mult_norm_LR_simul_CI Example:")
    print(f"True beta: {beta_true}")
    alpha = 0.05
    ci, method = mult_norm_lr_simul_ci(X, y, alpha=alpha)
    print(f"Simultaneous confidence intervals using method: {method}")
    for i, (lower, upper) in enumerate(ci):
        # print(f"CI for beta_{i}: ({lower:.4f}, {upper:.4f}) --> upper - lower = {upper - lower:.4f}")
        print(f"{100*(1 - alpha)}% CI for beta_{i}: ({lower:.4f}, {upper:.4f})")

    # Example 4: Mult_norm_LR_CR
    print("\n4. Mult_norm_LR_CR Example:")
    # Test for the first two coefficients
    C = np.array([
        [1, 0, 0, 0],  # beta_0
        [0, 1, 0, 0]  # beta_1
    ])
    alpha = 0.05
    cr = mult_norm_lr_cr(X, y, C, alpha=alpha)
    print(f"Center of the confidence region: {cr['center']}")
    print(f"Shape matrix of the confidence region:\n{cr['shape_matrix']}")
    print(f"Radius squared of the confidence region: {cr['radius_squared']:.4f}")

    # Example 5: Mult_norm_LR_is_in_CR
    print("\n5. Mult_norm_LR_is_in_CR Example:")
    # Check if the true coefficients are inside the confidence region
    c0 = np.array([beta_true[0], beta_true[1]])
    is_in_cr = mult_norm_lr_is_in_cr(X, y, C, c0, alpha=alpha)
    print(f"Is the true coefficient vector {c0} inside the confidence region? {'Yes' if is_in_cr else 'No'}")

    # Check if a different vector is inside the confidence region
    c0 = np.array([2, 1])
    is_in_cr = mult_norm_lr_is_in_cr(X, y, C, c0, alpha=alpha)
    print(f"Is the vector {c0} inside the confidence region? {'Yes' if is_in_cr else 'No'}")

    # Example 6: Mult_norm_LR_test_general
    print("\n6. Mult_norm_LR_test_general Example:")
    # Test the null hypothesis H0: C*beta = c0
    # True coefficients
    c0 = np.array([beta_true[0], beta_true[1]])
    test_result = mult_norm_lr_test_general(X, y, C, c0, alpha=alpha)
    print(f"Test result for H0: C*beta = c0 where c0 = {c0}")
    print(f"p-value: {test_result['p_value']:.4f} > {alpha}")
    print(f"F-statistic: {test_result['F_stat']:.4f}")
    print(f"Critical value: {test_result['f_critical']:.4f}")
    print(f"Decision: {'Reject H0' if test_result['reject'] else 'Fail to reject H0'}")
    print("")
    # Test false hypothesis
    c0 = np.array([2, 1])
    test_result = mult_norm_lr_test_general(X, y, C, c0, alpha=alpha)
    print(f"Test result for H0: C*beta = c0 where c0 = {c0}")
    print(f"p-value: {test_result['p_value']:.4f} < {alpha}")
    print(f"F-statistic: {test_result['F_stat']:.4f}")
    print(f"Critical value: {test_result['f_critical']:.4f}")
    print(f"Decision: {'Reject H0' if test_result['reject'] else 'Fail to reject H0'}")

    # Example 7: Mult_norm_LR_test_comp
    print("\n7. Mult_norm_LR_test_comp Example:")
    # Test if beta_2 = 0 and beta_3 = 0
    j_indices = [2, 3]
    test_result = mult_norm_lr_test_comp(X, y, j_indices, alpha=alpha)
    print(f"Test result for H0: beta_j = 0 for j in {j_indices}")
    print(f"p-value: {test_result['p_value']:.4f} < {alpha}")
    print(f"F-statistic: {test_result['F_stat']:.4f}")
    print(f"Critical value: {test_result['f_critical']:.4f}")
    print(f"Decision: {'Reject H0' if test_result['reject'] else 'Fail to reject H0'}")
    
    # Example 8: Mult_norm_LR_test_linear_reg
    print("\n8. Mult_norm_LR_test_linear_reg Example:")
    # Test if all coefficients (except intercept) are equal to 0
    test_result = mult_norm_lr_test_linear_reg(X, y, alpha=alpha)
    print(f"Test result for H0: beta_1 = beta_2 = ... = beta_k = 0")
    print(f"p-value: {test_result['p_value']:.4f} < {alpha}")
    print(f"F-statistic: {test_result['F_stat']:.4f}")
    print(f"Critical value: {test_result['f_critical']:.4f}")
    print(f"Decision: {'Reject H0' if test_result['reject'] else 'Fail to reject H0'}")

    # Example 9: Mult_norm_LR_pred_CI
    print("\n9. Mult_norm_LR_pred_CI Example:")
    # Create a new design matrix for predictions
    D = np.array([
        [1, -1, -1, -1],  # Prediction 1
        [1, 0, 0, 0],   # Prediction 2
        [1, 1, 1, 1]    # Prediction 3
    ])
    
    # True predicted values
    true_preds = D @ beta_true

    results = mult_norm_lr_pred_ci(X, y, D, alpha=alpha, method="best")
    print("Method used:", results['method'])    
    for i in range(D.shape[0]):
        
        lower = results['lower'][i]
        upper = results['upper'][i]
        
        print(f"Prediction {i+1}: True value = {true_preds[i]:.4f}, CI = ({lower:.4f}, {upper:.4f})")


In [54]:
def main():
    print("\n" + "*" * 80)
    print(" ANOVA and Linear Regression Toolbox Demo ".center(80, "*"))
    print("*"*80)

    # Run ANOVA examples
    run_anova_examples()

    # Run regression examples
    run_regression_examples()

In [55]:
if __name__ == "__main__":
    main()


********************************************************************************
******************* ANOVA and Linear Regression Toolbox Demo *******************
********************************************************************************

=========================== ANOVA Functions Examples ===========================

1. ANOVA1_partition_tss Example:
Total Sum of Squares (ss_total): 253.2957
Within-group Sum of Squares (ss_w): 146.7128
Between-group Sum of Squares (ss_b): 106.5830
Verification: ss_total = ss_w + ss_b: 253.2957 = 253.2957

2. ANOVA1_test_equality Example:

ANOVA Table
--------------------------------------------------------------------------------
Source          df         SS              MS              F              
--------------------------------------------------------------------------------
Between         2          106.5830        53.2915         15.2559        
Within          42         146.7128        3.4932         
Total           44         253.